<a href="https://colab.research.google.com/github/Stmark1/Finite_Element_Metoth/blob/main/M%C3%A9todo_de_m%C3%ADnimos_cuadrados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metodo de minimos cuadrados
Este programa resuelve problemas de estructuras.

Se pueden usar para resolver n numeros de elementos en la estructura.

## Problema de ejemplo a resolver
Determine los desplazamientos y fuerzas que actuan en cada eslabón de la estructura mostrada en la figura. El área de sección transversal de los elementos 1 y 2 es $40 cm^2$, para los elementos 3 y 4 es $30 cm^2$, y para el elemento 5 es $20 cm^2$. Los primeros 4 elementos están fabricados con un material con $E = 200 GPa$ y el ultimo elemento con un material de $E=70GPa$. Cada nodo tiene 2 grados de libertar y la carga P es de $150kN$.

|Nodo|X(m)|Y(m)|             |N° Elemento|Nodos|
|----|----|----|-------------|-----------|-----|
| 1  | 0  | 0  |             | 1         | 1-2 |
| 1  | 1.5| 3.5|             | 2         | 2-4 |
| 3  | 0  | 5  |             | 3         | 1-3 |
| 4  | 5  | 5  |             | 4         | 3-4 |
|    |    |    |             | 5         | 2-3 |



Tenemos 2 grados de libertad por cada nodo.

In [ ]:
# Codigo para general la imagen del armazón


##Importamos las librerias que vamos a utilizar
**Numpy** para trabajar con matrices.

**Reduce** una función que aplica otra hasta quedar con un elemento.

In [ ]:
import numpy as np
from functools import reduce

## Creamos la clase Elemento
Dentro de esta clase utilizamos el constructor $\_\_init\_\_$ para que almacene los valores que ocupamos para realizar cada operación.

Luego definimos los metodos **mdr** y **grados_l** para obtener los componentes que ocuparemos despues.

Datos ocupados:

* **Modulo de young** 
* **Area**
* **Cordenadas del punto inicial**
* **Cordenadas del punto final**
* **Nodo inicial**
* **Nodo final**

## Matriz de rigidez para cada elemento
$$\frac{EA}{L}\begin{pmatrix}
l^2_{s} & l_{s}m_{s} & -l^2_{s} & -l_{s}m_{s}\\
l_{s}m_{s} & m^2_{s} & -l_{s}m_{s} & -m^2_{s}\\
-l^2_{s} & -l_{s}m_{s} & l^2_{s} & l_{s}m_{s}\\
-l_{s}m_{s} & -m^2_{s} & l_{s}m_{s} & m^2_{s}\\
\end{pmatrix}\begin{pmatrix}
u_{1}\\
v_{1}\\
u_{2}\\
v_{2}\\
\end{pmatrix}=\begin{pmatrix}
F_{1x}\\
F_{1y}\\
F_{2x}\\
F_{2y}\\
\end{pmatrix}$$

In [ ]:
class Elemento():
  def __init__(self, cord1, cord2, e, a, inicio, final):

    # Cordenadas de los nodos del elemento
    self.cord1 = cord1
    self.cord2 = cord2

    #Propiedades del elemento
    self.e = e
    self.a = a

    # Nodos en los extremos
    self.inicio = inicio
    self.final = final
  
  # Definimos la operación que devuelva la matriz de rigidez
  def mdr(self):

    # Longitud del elemento
    longitud = ((self.cord2[0]-self.cord1[0])**2 + (self.cord2[1]-self.cord1[1])**2)**0.5
    
    ls = (self.cord2[0]-self.cord1[0]) / longitud
    ms = (self.cord2[1]-self.cord1[1]) / longitud
    
    eal = (self.e*self.a) / longitud

    matriz = np.array(
        [
            [ls**2, ls*ms, -(ls**2), -(ls*ms)],
            [ls*ms, ms**2, -(ls*ms), -(ms**2)],
            [-(ls**2), -(ls*ms), ls**2, ls*ms],
            [-(ls*ms), -(ms**2), ls*ms, ms**2]
        ]
    )

    matriz_rigidez = matriz * eal

    return matriz_rigidez

  # Definimos los grados de libertad que tiene el elemento
  def grados_l(self):

      ui = (self.inicio * 2) - 1
      vi = self.inicio * 2
      uf = (self.final * 2) - 1
      vf = self.final * 2

      grados = [ui, vi, uf, vf]

      return grados

## Creamos la clase Elemento con Resultados
Utilizamos la propiedad de herencia que existe dentro de python, de esta manera conseguimos no volver a escribir el codigo que utilizamos previamente.

Para eso dentro del *def \_\_init\_\_* llamamos al constructor de la clase padre, en este caso a Elemento.

In [ ]:
class Elemento_R(Elemento):

  def __init__(self, cord1, cord2, e, a, inicio, final, v_d):
    # Lamamos al constructor de la clase padre
    Elemento.__init__(self, cord1, cord2, e, a, inicio, final)
    
    self.vector_desplazamiento = v_d

    # Calculamos la longitud
    self.longitud = ((self.cord2[0] - self.cord1[0])**2 + (self.cord2[1] - self.cord1[1])**2)**0.5

    # Calcular ls y ms
    self.ls = (self.cord2[0] - self.cord1[0]) / self.longitud
    self.ms = (self.cord2[1] - self.cord1[1]) / self.longitud

    grados = self.grados_l()

    # Transformando el los grados de libertad a indice empezando en 0
    self.ui = grados[0] - 1
    self.vi = grados[1] - 1
    self.uf = grados[2] - 1
    self.vf = grados[3] - 1

    # Encontrando
    self.d1 = (self.ls * self.vector_desplazamiento[self.ui]) + (self.ms * self.vector_desplazamiento[self.vi])
    self.d2 = (self.ls * self.vector_desplazamiento[self.uf]) + (self.ms * self.vector_desplazamiento[self.vf])

  # Calcular el desplazamiento en algun punto a lo largo del elemento
  def des(self, s):
    if s <= self.longitud and s >= self.longitud:
      us = (((self.longitud + s) / s) * self.d1) + ((s / self.longitud) * self.d2)
      return us

    else:
      return print("Numero fuera del tamaño del elemento")

  # Calcular la deformación unitaria, es adimencional
  def def_unitaria(self):
    epsilon = (1 / self.longitud) * (-self.d1 + self.d2)
    return epsilon
  
  # Calculo del esfuerzo en el elemento
  def esfuerzo(self):
    sigma = self.e * self.def_unitaria()
    return sigma
  
  # Obteniendo la fuerza que existe en el elemento
  def fuerza(self):
    f = self.esfuerzo() * self.a
    return f
  
  # Función para inprimir en texto los valores importantes
  def resultados(self):
    epsilon = self.def_unitaria()
    sigma = self.esfuerzo()
    f = self.fuerza()
    resultados = print(f"El valor de la longitud es: {self.longitud:.4f} m\nEl valor del esfuerzo Unitario es: {epsilon:.6f}\nEl valor de la deformación unitaria es: {sigma:.4f} Pa\nEl valor de la fuerza es: {f:.4f} N")

    return resultados

## Matriz de endamble
En esta matriz es donde se van a ir colocando los valores de la matriz de rifides de manera que concuerden los numeros fila-columna con los valores de los grados de libertad, ya que tenemos una matriz cuadrada $u_{1}, v_{1}, u_{2}, v_{2}, ..., u_{n}, v_{n}$

In [ ]:
# Añadir los elementos usando los grados de libertad

def matriz_ensamble(elemento, tamaño_matriz):

  # Creamos la matriz donde agregar los datos de acuerdo a sus grados de libertad
  matriz_base = np.zeros((tamaño_matriz, tamaño_matriz), dtype=float)

  # Almacenamos los grados de libertad utilizando el metodo previamente definido en la clase
  ui = elemento.grados_l()[0]
  vi = elemento.grados_l()[1]
  uf = elemento.grados_l()[2]
  vf = elemento.grados_l()[3]

  ui = ui - 1
  vi = vi - 1
  uf = uf - 1
  vf = vf - 1

  # Revisamos los 16 lugares de la matriz de rigides
  for i in range(tamaño_matriz):

    for j in range(tamaño_matriz):

      if i == ui and j == ui:
        matriz_base[i, j] = elemento.mdr()[0, 0]

      elif i == ui and j == vi:
        matriz_base[i, j] = elemento.mdr()[0, 1]

      elif i == ui and j == uf:
        matriz_base[i, j] = elemento.mdr()[0, 2]

      elif i == ui and j == vf:
        matriz_base[i, j] = elemento.mdr()[0, 3]

      elif i == vi and j == ui:
        matriz_base[i, j] = elemento.mdr()[1, 0]

      elif i == vi and j == vi:
        matriz_base[i, j] = elemento.mdr()[1, 1]

      elif i == vi and j == uf:
        matriz_base[i, j] = elemento.mdr()[1, 2]

      elif i == vi and j == vf:
        matriz_base[i, j] = elemento.mdr()[1, 3]

      elif i == uf and j == ui:
        matriz_base[i, j] = elemento.mdr()[2, 0]

      elif i == uf and j == vi:
        matriz_base[i, j] = elemento.mdr()[2, 1]

      elif i == uf and j == uf:
        matriz_base[i, j] = elemento.mdr()[2, 2]

      elif i == uf and j == vf:
        matriz_base[i, j] = elemento.mdr()[2, 3]

      elif i == vf and j == ui:
        matriz_base[i, j] = elemento.mdr()[3, 0]

      elif i == vf and j == vi:
        matriz_base[i, j] = elemento.mdr()[3, 1]

      elif i == vf and j == uf:
        matriz_base[i, j] = elemento.mdr()[3, 2]

      elif i == vf and j == vf:
        matriz_base[i, j] = elemento.mdr()[3, 3]

  return matriz_base

## Generando los elementos
Ahora creamos una función para generar elementos.

Utilizamos compreensión de lista y de diccionarios.

Seleccionamos que tipos de clase vamos a usar para generar la clase usando r="r"

Damos un valor nulo por default para r y v_d. Recordando que los valores default se añaden despues.

In [ ]:
def generador_elementos(elementos, r=None, v_d=None):
  
  # Creando la lista de los nombres de los elementos
  nombres = ["elemento" + str(i + 1) for i in range(len(elementos))]
  
  if r == None:
    # Unir los valores y generar los elementos
    dic_elementos = {nombre:Elemento(*valor) for (nombre, valor) in zip(nombres, elementos)}
    return dic_elementos
  
  elif r == "r":
    dic_elementos = {nombre:Elemento_R(*valor, v_d) for (nombre, valor) in zip(nombres, elementos)}
    return dic_elementos

## Aqui vamos a agregar los valores de los elementos

In [ ]:
# Lista de elementos en el problema con sus propiedades
# Es una lista de listas con:
# (cordenada del nodo inicial), (cordenada del nodo final), modulo de young, área, nodo inicial, nodo final

"""
elementos = [
             [("x1", "y1"), ("x2", "y2"), "e", "a", "inicio", "final"],
             [("x1", "y1"), ("x2", "y2"), "e", "a", "inicio", "final"],
             [("x1", "y1"), ("x2", "y2"), "e", "a", "inicio", "final"]
]
"""
numero_nodos = 4

elementos = [
             [(0, 0), (1.5, 3.5), 200000000000, 0.004, 1, 2],
             [(1.5, 3.5), (5, 5), 200000000000, 0.004, 2, 4],
             [(0, 0), (0, 5), 200000000000, 0.003, 1, 3],
             [(0, 5), (5, 5), 200000000000, 0.003, 3, 4],
             [(1.5, 3.5), (0, 5), 70000000000, 0.002, 2, 3]
            ]

# Vector carga, x1, y1, x2, y2, ... xn, yn *Por cada grada de libertad
vector_carga = np.array([0, 0, 0, -150000, 0, 0, 0, 0])

## Creamos los elementos con la clase y la lista de datos de los elementos
Asignamos el diccionario generado por nuestra función a dic_elementos

Utilizamos el ciclo *for* para revisar que los datos sean coerentes

In [ ]:
# Revisar los valores de la matriz de regides de cada elemento
dic_elementos = generador_elementos(elementos)

for i in dic_elementos.values():
  print(i.mdr())
  print(i.grados_l())

[[ 3.26002178e+07  7.60671749e+07 -3.26002178e+07 -7.60671749e+07]
 [ 7.60671749e+07  1.77490075e+08 -7.60671749e+07 -1.77490075e+08]
 [-3.26002178e+07 -7.60671749e+07  3.26002178e+07  7.60671749e+07]
 [-7.60671749e+07 -1.77490075e+08  7.60671749e+07  1.77490075e+08]]
[1, 2, 3, 4]
[[ 1.77490075e+08  7.60671749e+07 -1.77490075e+08 -7.60671749e+07]
 [ 7.60671749e+07  3.26002178e+07 -7.60671749e+07 -3.26002178e+07]
 [-1.77490075e+08 -7.60671749e+07  1.77490075e+08  7.60671749e+07]
 [-7.60671749e+07 -3.26002178e+07  7.60671749e+07  3.26002178e+07]]
[3, 4, 7, 8]
[[ 0.0e+00  0.0e+00 -0.0e+00 -0.0e+00]
 [ 0.0e+00  1.2e+08 -0.0e+00 -1.2e+08]
 [-0.0e+00 -0.0e+00  0.0e+00  0.0e+00]
 [-0.0e+00 -1.2e+08  0.0e+00  1.2e+08]]
[1, 2, 5, 6]
[[ 1.2e+08  0.0e+00 -1.2e+08 -0.0e+00]
 [ 0.0e+00  0.0e+00 -0.0e+00 -0.0e+00]
 [-1.2e+08 -0.0e+00  1.2e+08  0.0e+00]
 [-0.0e+00 -0.0e+00  0.0e+00  0.0e+00]]
[5, 6, 7, 8]
[[ 32998316.45537223 -32998316.45537223 -32998316.45537223
   32998316.45537223]
 [-32998316.455

## Sumamos todas las matrices para crear la **matriz de rigidez global**

In [ ]:
# Aplicamos la operación matriz_ensamble() a las matrices de rigides y los almacenamos en una lista
arreglos = [matriz_ensamble(matriz, numero_nodos * 2) for matriz in generador_elementos(elementos).values()]

# Utilizando reduce() se suman las matrices
matriz_final = reduce(lambda x, y: x+y, arreglos)

print(matriz_final)

[[ 3.26002178e+07  7.60671749e+07 -3.26002178e+07 -7.60671749e+07
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.60671749e+07  2.97490075e+08 -7.60671749e+07 -1.77490075e+08
   0.00000000e+00 -1.20000000e+08  0.00000000e+00  0.00000000e+00]
 [-3.26002178e+07 -7.60671749e+07  2.43088609e+08  1.19136033e+08
  -3.29983165e+07  3.29983165e+07 -1.77490075e+08 -7.60671749e+07]
 [-7.60671749e+07 -1.77490075e+08  1.19136033e+08  2.43088609e+08
   3.29983165e+07 -3.29983165e+07 -7.60671749e+07 -3.26002178e+07]
 [ 0.00000000e+00  0.00000000e+00 -3.29983165e+07  3.29983165e+07
   1.52998316e+08 -3.29983165e+07 -1.20000000e+08  0.00000000e+00]
 [ 0.00000000e+00 -1.20000000e+08  3.29983165e+07 -3.29983165e+07
  -3.29983165e+07  1.52998316e+08  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.77490075e+08 -7.60671749e+07
  -1.20000000e+08  0.00000000e+00  2.97490075e+08  7.60671749e+07]
 [ 0.00000000e+00  0.00000000e+00 -7.60671749e+07 -3.26002178e+07
   

## No olvidar colocar los vaolres frontera
Los nodos que no se mueven

In [ ]:
# Agregar valores de frontera

"""def frontera_valores(nodos_fijos)
  nodos_fijos[]"""

# Colocamos primero la fila en 0
# Posteriormente colocamos el 1 en la diagonal

# Primer nodo fijo
matriz_final[0] = 0
matriz_final[1] = 0

matriz_final[0,0] = 1
matriz_final[1,1] = 1

#Segundo nodo fijo
matriz_final[6] = 0
matriz_final[7] = 0

matriz_final[6,6] = 1
matriz_final[7,7] = 1

# Revisió de los valores 
for row in matriz_final:
  print(row)

[1. 0. 0. 0. 0. 0. 0. 0.]
[0. 1. 0. 0. 0. 0. 0. 0.]
[-3.26002178e+07 -7.60671749e+07  2.43088609e+08  1.19136033e+08
 -3.29983165e+07  3.29983165e+07 -1.77490075e+08 -7.60671749e+07]
[-7.60671749e+07 -1.77490075e+08  1.19136033e+08  2.43088609e+08
  3.29983165e+07 -3.29983165e+07 -7.60671749e+07 -3.26002178e+07]
[ 0.00000000e+00  0.00000000e+00 -3.29983165e+07  3.29983165e+07
  1.52998316e+08 -3.29983165e+07 -1.20000000e+08  0.00000000e+00]
[ 0.00000000e+00 -1.20000000e+08  3.29983165e+07 -3.29983165e+07
 -3.29983165e+07  1.52998316e+08  0.00000000e+00  0.00000000e+00]
[0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 1.]


## Finalmente invertimos la matriz
Usamos $np.linalg.inv()$

Esta función invierta la matriz si es posible matematicamente.

In [ ]:
#invertimos la matriz
matriz_final_inv = np.linalg.inv(matriz_final)

for row in matriz_final_inv:
  print(row)

[ 1.00000000e+00  5.18104078e-16  1.33095046e-24  3.11611780e-24
  2.67501718e-24 -3.52361629e-24 -6.23276243e-15 -8.00324366e-17]
[-2.87570530e-18  1.00000000e+00 -4.20459463e-25 -1.06295392e-25
 -1.54240313e-24 -5.45015142e-25  2.61519335e-16  8.29555756e-18]
[-6.61778310e-02 -3.66177831e-01  6.35374200e-09 -3.59302425e-09
  1.76469077e-09 -1.76469077e-09  1.06617783e+00  3.66177831e-01]
[ 3.66177831e-01  1.06617783e+00 -3.59302425e-09  6.35374200e-09
 -1.76469077e-09  1.76469077e-09 -3.66177831e-01 -6.61778310e-02]
[-7.67057378e-02 -7.67057378e-02  1.76469077e-09 -1.76469077e-09
  7.48104736e-09  8.52285976e-10  1.07670574e+00  7.67057378e-02]
[ 7.67057378e-02  1.07670574e+00 -1.76469077e-09  1.76469077e-09
  8.52285976e-10  7.48104736e-09 -7.67057378e-02 -7.67057378e-02]
[0. 0. 0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 0. 1.]


## Para resolver nada mas multiplicamos la inversa por el vector carga
$$\frac{EA}{L}\begin{pmatrix}
l^2_{s} & l_{s}m_{s} & -l^2_{s} & -l_{s}m_{s}\\
l_{s}m_{s} & m^2_{s} & -l_{s}m_{s} & -m^2_{s}\\
-l^2_{s} & -l_{s}m_{s} & l^2_{s} & l_{s}m_{s}\\
-l_{s}m_{s} & -m^2_{s} & l_{s}m_{s} & m^2_{s}\\
\end{pmatrix}^{-1}\begin{pmatrix}
F_{1x}\\
F_{1y}\\
F_{2x}\\
F_{2y}\\
\end{pmatrix}=\begin{pmatrix}
u_{1}\\
v_{1}\\
u_{2}\\
v_{2}\\
\end{pmatrix}$$

In [ ]:
# Multiplicación de la matriz inversa por el vector carga nos da el resultado del vector desplazamiento

vector_desplazamiento = np.dot(matriz_final_inv, vector_carga)

print(vector_desplazamiento)

[-4.67417669e-19  1.59443088e-20  5.38953638e-04 -9.53061301e-04
  2.64703615e-04 -2.64703615e-04  0.00000000e+00  0.00000000e+00]


## Generamos los resultados
Ya que tenemos el vector de desplazamiento podemos calcular los valores dependientes de ella.
* Deformación unitaria
* Esfuerzo
* Fuerza
* Desplazamiento en x distancia

In [ ]:
# Utilizamos la misma función, pero con los valores opcionales llenos
dic_elementos_r = generador_elementos(elementos, r="r", v_d=vector_desplazamiento)

## Mostramos los resultados
Utilizamos el metodo **Elemento_R.resultados()**

Utilizando el ciclo *for* para imprimir los resultados por elemento.

In [ ]:
# Ciclo para imprimir todos los resultados

for key in dic_elementos_r.keys():
  print(key)
  dic_elementos_r[key].resultados()

elemento1
El valor de la longitud es: 3.8079 m
El valor del esfuerzo Unitario es: -0.000174
El valor de la deformación unitaria es: -34859090.9686 Pa
El valor de la fuerza es: -139436.3639 N
elemento2
El valor de la longitud es: 3.8079 m
El valor del esfuerzo Unitario es: -0.000031
El valor de la deformación unitaria es: -6299941.8216 Pa
El valor de la fuerza es: -25199.7673 N
elemento3
El valor de la longitud es: 5.0000 m
El valor del esfuerzo Unitario es: -0.000053
El valor de la deformación unitaria es: -10588144.5983 Pa
El valor de la fuerza es: -31764.4338 N
elemento4
El valor de la longitud es: 5.0000 m
El valor del esfuerzo Unitario es: -0.000053
El valor de la deformación unitaria es: -10588144.5983 Pa
El valor de la fuerza es: -31764.4338 N
elemento5
El valor de la longitud es: 2.1213 m
El valor del esfuerzo Unitario es: 0.000321
El valor de la deformación unitaria es: 22460846.5370 Pa
El valor de la fuerza es: 44921.6931 N
